## RAG向けにLLMを指示チューニングする

In [1]:
!pip install datasets transformers[torch,sentencepiece] trl peft bitsandbytes --q

In [2]:
from transformers.trainer_utils import set_seed

set_seed(42)

In [3]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)
%cd "/content/drive/Othercomputers/マイ MacBook Air/GenerativeAI_apps/13.RAGシステムの実装"

Mounted at /content/drive
/content/drive/Othercomputers/マイ MacBook Air/GenerativeAI_apps/13.RAGシステムの実装


### データセットの準備

In [4]:
from datasets import load_dataset

dataset = load_dataset(
    "llm-book/aio-retriever", trust_remote_code=True
)

print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers', 'passages', 'positive_passage_indices', 'negative_passage_indices'],
        num_rows: 22335
    })
    validation: Dataset({
        features: ['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers', 'passages', 'positive_passage_indices', 'negative_passage_indices'],
        num_rows: 1000
    })
})


In [5]:
from pprint import pprint

pprint(dataset["validation"][0])

{'answers': ['ジェット団'],
 'competition': '第2回AI王',
 'negative_passage_indices': [1,
                              2,
                              4,
                              6,
                              8,
                              9,
                              10,
                              11,
                              13,
                              14,
                              15,
                              16,
                              17,
                              18,
                              19,
                              20,
                              21,
                              23,
                              24,
                              25,
                              26,
                              27,
                              28,
                              29,
                              30,
                              31,
                              32,
                              33,
     

In [6]:
from typing import Any

def filter_example(example: dict[str, Any], max_passgages: int = 3) -> bool:
    """上位max_passages件のパッセージに正例が含まれていない場合は除外"""
    if len(example["positive_passage_indices"]) == 0:
        return False
    if example["positive_passage_indices"][0] >= max_passgages:
        return False
    return True

dataset = dataset.filter(filter_example)

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers', 'passages', 'positive_passage_indices', 'negative_passage_indices'],
        num_rows: 13951
    })
    validation: Dataset({
        features: ['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers', 'passages', 'positive_passage_indices', 'negative_passage_indices'],
        num_rows: 637
    })
})

In [15]:
def process_example(
    example: dict[str, Any], max_passages: int = 3
) -> dict[str, Any]:
    """質問、パッセージ、正解の組からプロンプトを作成し、会話データに変換"""

    # exampleから必要な情報を取得
    question = example["question"]
    answer = example["answers"][0]
    passages = [p["text"] for p in example["passages"]]

    # max_passages件のパッセージを選択
    passages = passages[:max_passages]

    messages: list[dict[str, str]] = []
    # プロンプトとパッセージをユーザのメッセージとして会話データに追加
    prompt_text = "".join(
        [
            "あなたには今からクイズに答えてもらいます。",
            "問題を与えますので、その解答のみを簡潔に出力してください。\n",
            "また解答の参考になりうるテキストを与えます。",
            "解答を含まない場合もあるのでその場合は無視してください。\n\n",
            "---\n",
            "\n\n".join(passages),
            "\n---\n\n",
            f"問題: {question}",
        ]
    )
    messages.append({"role": "user", "content": prompt_text})
    # LLMが出力すべき内容（クイズ問題の答え）を会話データに追加
    messages.append({"role": "assistant", "content": answer})

    # 会話データを事例の"messages"フィールドに追加
    example["messages"] = messages
    return example

detaset = dataset.map(
    process_example, remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/13951 [00:00<?, ? examples/s]

KeyError: 'question'

In [16]:
pprint(dataset["validation"][0])

{'messages': [{'content': 'あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。\n'
                          'また解答の参考になりうるテキストを与えます。解答を含まない場合もあるのでその場合は無視してください。\n'
                          '\n'
                          '---\n'
                          'ニューヨークのウエスト・サイド。午後5時。ポーランド系アメリカ人の少年非行グループ「ジェッツ」(ジェット団)と、新参のプエルトリコ系アメリカ人の少年非行グループ「シャークス」(シャーク団)は、なわばりを巡って対立している。今日も2グループの間で争いが起きるが警官の呼子笛の音に止められる(“Prologue”「プロローグ」)。クラプキ巡査とシュランク警部補が現れて少年たちに説教をして帰っていく。ジェッツのリーダー・リフはシャークスとの関係をはっきりさせるために決闘しようと言い出し、ジェッツのメンバーが賛成する。ついては決闘についての取り決めをシャークスとする必要があり、リフは自分の副官にトニーを選ぶ。メンバーは初めトニーはもう抜けたと反対するが、リフは(海兵隊のように)「一度ジェッツになったら死ぬまでジェッツだ」と歌う。\n'
                          '\n'
                          '『ウエストサイド物語』(ウエストサイドものがたり)は、宝塚歌劇団によるミュージカル作品。ブロードウェイ・ミュージカルの傑作『ウエストサイド物語』の日本での上演の一つである。\n'
                          '\n'
                          '『ウエスト・サイド物語』(ウエスト・サイドものがたり、West Side '
                          'Story)は、アーサー・ローレンツ脚本、レナード・バーンスタイン音楽、スティーヴン・ソンドハイム歌詞のブロードウェイ・ミュージカル。原案ジェローム・ロビンズ。1957年初演。『ウエスト・サイド・ストーリー』とも

### トークナイザとモデルの準備

In [17]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

base_model_name = "llm-book/Swallow-7b-hf-oasst1-21k-ja"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

In [18]:
tokenizer.chat_template

"{%- for message in messages %}\n{%- if message['role'] == 'user' %}\n{{ bos_token + 'ユーザ：' + message['content'] + eos_token }}\n{%- elif message['role'] == 'assistant' %}\n{{ bos_token + 'アシスタント：'  + message['content'] + eos_token }}\n{%- endif %}\n{% if loop.last and add_generation_prompt %}\n{{ bos_token + 'アシスタント：' }}\n{%- endif %}\n{% endfor %}"

In [26]:
# トークナイザーでテンプレートを適用
template_output = tokenizer.apply_chat_template(
    dataset["validation"][0]["messages"][:-1],
    tokenize=False,  # tokenizeはFalseが正しい（質問のtokenizerはtypo）
    add_generation_prompt=True
)

print(template_output)

<s>ユーザ：あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。
また解答の参考になりうるテキストを与えます。解答を含まない場合もあるのでその場合は無視してください。

---
ニューヨークのウエスト・サイド。午後5時。ポーランド系アメリカ人の少年非行グループ「ジェッツ」(ジェット団)と、新参のプエルトリコ系アメリカ人の少年非行グループ「シャークス」(シャーク団)は、なわばりを巡って対立している。今日も2グループの間で争いが起きるが警官の呼子笛の音に止められる(“Prologue”「プロローグ」)。クラプキ巡査とシュランク警部補が現れて少年たちに説教をして帰っていく。ジェッツのリーダー・リフはシャークスとの関係をはっきりさせるために決闘しようと言い出し、ジェッツのメンバーが賛成する。ついては決闘についての取り決めをシャークスとする必要があり、リフは自分の副官にトニーを選ぶ。メンバーは初めトニーはもう抜けたと反対するが、リフは(海兵隊のように)「一度ジェッツになったら死ぬまでジェッツだ」と歌う。

『ウエストサイド物語』(ウエストサイドものがたり)は、宝塚歌劇団によるミュージカル作品。ブロードウェイ・ミュージカルの傑作『ウエストサイド物語』の日本での上演の一つである。

『ウエスト・サイド物語』(ウエスト・サイドものがたり、West Side Story)は、アーサー・ローレンツ脚本、レナード・バーンスタイン音楽、スティーヴン・ソンドハイム歌詞のブロードウェイ・ミュージカル。原案ジェローム・ロビンズ。1957年初演。『ウエスト・サイド・ストーリー』とも呼ばれる。シェイクスピアの戯曲『ロミオとジュリエット』に着想し、当時のニューヨークの社会的背景を織り込みつつ、ポーランド系アメリカ人とプエルトリコ系アメリカ人との2つの異なる少年非行グループの抗争の犠牲となる若い男女の2日間の恋と死までを描く。1961年と2021年に映画化された。
---

問題: 映画『ウエスト・サイド物語』に登場する2つの少年グループといえば、シャーク団と何団?</s><s>アシスタント：


In [27]:
dataset["validation"][0]["messages"]

[{'content': 'あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。\nまた解答の参考になりうるテキストを与えます。解答を含まない場合もあるのでその場合は無視してください。\n\n---\nニューヨークのウエスト・サイド。午後5時。ポーランド系アメリカ人の少年非行グループ「ジェッツ」(ジェット団)と、新参のプエルトリコ系アメリカ人の少年非行グループ「シャークス」(シャーク団)は、なわばりを巡って対立している。今日も2グループの間で争いが起きるが警官の呼子笛の音に止められる(“Prologue”「プロローグ」)。クラプキ巡査とシュランク警部補が現れて少年たちに説教をして帰っていく。ジェッツのリーダー・リフはシャークスとの関係をはっきりさせるために決闘しようと言い出し、ジェッツのメンバーが賛成する。ついては決闘についての取り決めをシャークスとする必要があり、リフは自分の副官にトニーを選ぶ。メンバーは初めトニーはもう抜けたと反対するが、リフは(海兵隊のように)「一度ジェッツになったら死ぬまでジェッツだ」と歌う。\n\n『ウエストサイド物語』(ウエストサイドものがたり)は、宝塚歌劇団によるミュージカル作品。ブロードウェイ・ミュージカルの傑作『ウエストサイド物語』の日本での上演の一つである。\n\n『ウエスト・サイド物語』(ウエスト・サイドものがたり、West Side Story)は、アーサー・ローレンツ脚本、レナード・バーンスタイン音楽、スティーヴン・ソンドハイム歌詞のブロードウェイ・ミュージカル。原案ジェローム・ロビンズ。1957年初演。『ウエスト・サイド・ストーリー』とも呼ばれる。シェイクスピアの戯曲『ロミオとジュリエット』に着想し、当時のニューヨークの社会的背景を織り込みつつ、ポーランド系アメリカ人とプエルトリコ系アメリカ人との2つの異なる少年非行グループの抗争の犠牲となる若い男女の2日間の恋と死までを描く。1961年と2021年に映画化された。\n---\n\n問題: 映画『ウエスト・サイド物語』に登場する2つの少年グループといえば、シャーク団と何団?',
  'role': 'user'},
 {'content': 'ジェット団', 'role': 'assistant'}]

### チューニング前のモデルの評価

In [32]:
from datasets import Dataset
from tqdm.notebook import tqdm
from transformers import PreTrainedModel
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

def evaluate(model: PreTrainedModel, dataset: Dataset) -> tuple[list[str], list[str], float]:
    """データセットの各問題に対するモデルの出力を評価し、正解率を算出"""
    pred_answers = []
    gold_answers = []
    num_correct = 0

    for example in tqdm(dataset):
        model_inputs = tokenizer.apply_chat_template(
            example["messages"][:-1],
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        input_length = model_inputs.shape[1]

        # モデルにプロンプトを入力し、出力を得る
        generated_ids = model.generate(
            model_inputs,
            max_new_tokens=32,
            do_sample=False,
            temperature=None,
            top_p=None
        )
        # モデルの出力から答えの部分を文字列として取り出す
        pred_answer = tokenizer.batch_decode(
            generated_ids[:, input_length:], skip_special_tokens=True,
        )[0]

        # 正解の文字列を取り出す
        gold_answer = example["messages"][-1]["content"]

        # モデルの答えと正解が一致していれば正解とカウント
        if pred_answer == gold_answer:
            num_correct += 1

        # モデルの答えと正解をそれぞれのリストに追加
        pred_answers.append(pred_answer)
        gold_answers.append(gold_answer)

    # 正解率を計算
    accuracy = num_correct / len(pred_answers)
    logger.info(f"Accuracy: {accuracy}")
    logger.info(f"Correct: {num_correct}")
    logger.info(f"Total: {len(pred_answers)}")

    return pred_answers, gold_answers, accuracy

In [33]:
pred_answer, gold_answers, accuracy = evaluate(model, dataset["validation"])

  0%|          | 0/637 [00:00<?, ?it/s]

INFO:__main__:Accuracy: 0.5196232339089482
INFO:__main__:Correct: 331
INFO:__main__:Total: 637


In [36]:
# モデルが予測した答えを表示
for pred_answer, gold_answer in zip(pred_answer[:20], gold_answers[:20]):
    print(f"正解: {gold_answer} / 予測: {pred_answer}")

正解: ジェット団 / 予測: ジェッツ
正解: コマイ / 予測: スケトウダラ
正解: START / 予測: START
正解: ニュートン / 予測: アイザック・ニュートン
正解: 天平文化 / 予測: 聖武天皇の時代に栄えた、東大寺正倉院や唐招提寺金堂など、中国・唐の影響を強く受け
正解: アメリカンリーグ / 予測: アメリカンリーグ
正解: 華道 / 予測: 池坊、草月流、小原流は、日本の伝統的な生け花の三大流派である。池坊は伝統
正解: ラストベルト / 予測: ラストベルト
正解: 天童市 / 予測: 天童市
正解: 医学部 / 予測: 安部公房は東京大学医学部出身。
正解: 村田珠光 / 予測: 山上宗二は、「侘び茶」の創始者として知られる室町時代の茶人である。彼は
正解: 23時 / 予測: 日本のテレビ業界で「プライムタイム」といえば、毎日19時から23時までの時間帯のことです。
正解: 佐々木彩夏 / 予測: 佐々木彩夏
正解: 早口言葉 / 予測: 英語で「タングツイスター」という言葉遊びは「早口言葉」です。
正解: 昭和基地 / 予測: 昭和基地
正解: 開口一番 / 予測: 「開口一番」
正解: マクベス / 予測: マクベス
正解: ニ長調 / 予測: ト短調
正解: 版籍奉還 / 予測: 版籍奉還
正解: IBS / 予測: IBS


### 指示チューニングの準備

In [48]:
tokenized_train_dataset = [
    tokenizer.apply_chat_template
     (example["messages"] )
     for example in dataset["train"]
]

In [38]:
from trl import DataCollatorForCompletionOnlyLM

bos = tokenizer.bos_token
collator = DataCollatorForCompletionOnlyLM(
    # ユーザとアシスタントそれぞれの発話開始文字列
    instruction_template=bos + "ユーザ：",
    response_template=bos + "アシスタント：",
    tokenizer=tokenizer
)

In [39]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(
    r=128,
    lora_alpha=128,
    lora_dropout=0.05,
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
)

model.enable_input_require_grads()
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 319,815,680 || all params: 7,149,785,088 || trainable%: 4.4731


In [40]:
import os
import wandb
from google.colab import userdata

def setup_wandb(project_name: str, run_name: str, config: str, job_type=None):
    # set up your API key
    try:
        WANDB_KEY = userdata.get('WANDB_API_KEY')
        wandb.login(key=WANDB_KEY)
        os.environ["WANDB_ENTITY"] = "y-hiroki-rad"
    except KeyError:
        raise EnvironmentError("WANDB_API_KEY is not set in the environment variables.")
    except Exception as e:
        print(f"Error logging into WandB: {e}")

    # Optional: Log models
    os.environ["WANDB_LOG_MODEL"] = "checkpoint"
    os.environ["WANDB_WATCH"] = "all"
    os.environ["WANDB_SILENT"] = "true"

    # Initialize the WandB run
    try:
        if config:
            wandb.init(project=project_name, name=run_name, config=config, job_type=job_type)
        elif:
            wandb.init(project=project_name, name=run_name, job_type=job_type)
        print(f"WandB run initialized: Project - {project_name}, Run - {run_name}")
    except Exception as e:
        print(f"Error initializing WandB run: {e}")

### 指示チューニングの実行

In [41]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir="RAG_IT_results",
    bf16=True,
    max_steps=100,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    logging_steps=10,
    save_steps=50,
    report_to="wandb"
)

In [50]:
trainer = Trainer(
    model,
    train_dataset=tokenized_train_dataset,
    data_collator=collator,
    args=training_args,
    tokenizer=tokenizer,
)

<ipython-input-50-82336111d8f9>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [44]:
setup_wandb(project_name="RAG_IT", run_name="fine-tuning", config=training_args)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: y-hiroki-rad. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


WandB run initialized: Project - RAG_IT, Run - fine-tuning


In [51]:
trainer.train()

Step,Training Loss
10,0.374100
20,0.169900
30,0.094200
40,0.144400
50,0.128100
60,0.123800
70,0.123200
80,0.163200
90,0.122100


wandb: Adding directory to artifact (./RAG_IT_results/checkpoint-50)... Done. 7.4s


Step,Training Loss
10,0.374100
20,0.169900
30,0.094200
40,0.144400
50,0.128100
60,0.123800
70,0.123200
80,0.163200
90,0.122100
100,0.122500


wandb: Adding directory to artifact (./RAG_IT_results/checkpoint-100)... Done. 5.7s


TrainOutput(global_step=100, training_loss=0.15656252920627595, metrics={'train_runtime': 3206.5041, 'train_samples_per_second': 0.499, 'train_steps_per_second': 0.031, 'total_flos': 5.182191345962189e+16, 'train_loss': 0.15656252920627595, 'epoch': 0.11467889908256881})

In [52]:
wandb.finish()

train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/grad_norm,▄▇▂▂█▃█▅▁▃
train/learning_rate,██▇▆▅▄▃▂▁▁
train/loss,█▃▁▂▂▂▂▃▂▂
total_flos,5.182191345962189e+16
train/epoch,0.11468
train/global_step,100
train/grad_norm,0.74278
train/learning_rate,0
train/loss,0.1225


### 指示チューニング後のモデルを使って評価

In [ ]:
pred_answers, gold_answers, accuracy = evaluate(
    model, dataset["validation"]
)

print(f"正解率：{accuracy:.1%}")

In [ ]:
# モデルが予測した解答を表示
for pred_answer, gold_answer in zip(
    pred_answers[:20], gold_answers[:20]
):
    print(f"正解: {gold_answer} / 予測: {pred_answer}")

### モデルの保存

In [53]:
from google.colab import userdata

HUGGINGFACE_TOKEN = userdata.get('HF_TOKEN_WRITE')
!huggingface-cli login --token $HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `LLM_new_token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLM_new_token`


In [55]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
)
checkpoint_path = "/content/drive/Othercomputers/マイ MacBook Air/GenerativeAI_apps/13.RAGシステムの実装/RAG_IT_results/checkpoint-100"
tuned_model = PeftModel.from_pretrained(base_model, checkpoint_path)

tuned_model = tuned_model.merge_and_unload()

repo_name = "hiroki-rad/Swallow-7b-hf-oasst1-21k-ja-aio-retriever"

tokenizer.push_to_hub(repo_name)
tuned_model.push_to_hub(repo_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.we

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hiroki-rad/Swallow-7b-hf-oasst1-21k-ja-aio-retriever/commit/d1838745413c0ecf9387b89e08313cef24b91c5d', commit_message='Upload LlamaForCausalLM', commit_description='', oid='d1838745413c0ecf9387b89e08313cef24b91c5d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hiroki-rad/Swallow-7b-hf-oasst1-21k-ja-aio-retriever', endpoint='https://huggingface.co', repo_type='model', repo_id='hiroki-rad/Swallow-7b-hf-oasst1-21k-ja-aio-retriever'), pr_revision=None, pr_num=None)